In [16]:
import os
%load_ext autoreload
%autoreload 2
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import json
import numpy as np
#import rpy2.robjects as robjects
#from rpy2.robjects import r, pandas2ri
#from rpy2.robjects.packages import importr
from matplotlib.ticker import MaxNLocator
from os.path import exists
import pickle
from scipy.stats import sem
import statistics
import scipy
from sklearn.metrics import mean_squared_error
from kneed import KneeLocator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
##### Save self finding steps in contingency game for each level #####

## Get no-movement action count of agent for each level
def get_self_finding_steps(game_type):
    files = glob.glob("../data/" + game_type + "/data_sf/*.json")

    lvl_count = 53 if 'change_agent_game' in game_type else 100

    if 'change_agent_game' in game_type:
        all_levels = {'steps': [], 'self_finding_steps': [], 'prop_selected_correctly': [], 'participant': [], 'level': []}
    else:
        all_levels = {'steps': [], 'self_finding_steps': [], 'participant': [], 'level': []}
    for i, file in enumerate(files):  # Get the data for each participant in that level
        data = json.load(open(file))

        for level in range(0, lvl_count):  # Data for each level
            selected_agents = data.get("selectedAgents")

            # Correctly clicked agents in a level
            correct_sf_level = [d for d in selected_agents if d['level'] == level and d['real_self'] == d['clicked_self']]

            if 'change_agent_game' in game_type:
                # Clicked agents in a level (regardless of whether they are correct or not)
                level_clicks = [d for d in selected_agents if d['level'] == level]
                if len(level_clicks) >= 1:
                    avg_sf_steps = [c['step'] % 7 if c['real_self'] == c['clicked_self'] else 7 for c in level_clicks]
                    #avg_sf_steps = [c['step'] % 7 for c in correct_sf_level]
                    all_levels['self_finding_steps'].append(statistics.mean(avg_sf_steps) if len(avg_sf_steps) > 0 else None)
                    all_levels['prop_selected_correctly'].append(statistics.mean([d['real_self'] == d['clicked_self'] for d in level_clicks]) if len(level_clicks) > 0 else None)
                    all_levels['steps'].append(data['data']['steps'][level])
                else:
                    all_levels['self_finding_steps'].append(None)
                    all_levels['prop_selected_correctly'].append(None)
                    all_levels['steps'].append(data['data']['steps'][level])
            else:
                if len(correct_sf_level) == 1:  # Clicked the correct agent in a level
                    all_levels['steps'].append(data['data']['steps'][level])
                    all_levels['self_finding_steps'].append(correct_sf_level[0]['step'])
                else:  # Did not click the correct agent in a level
                    all_levels['steps'].append(data['data']['steps'][level])
                    all_levels['self_finding_steps'].append(None)
            
            all_levels['participant'].append(i)
            all_levels['level'].append(level)

    df = pd.DataFrame(all_levels)
    df.to_csv('../stats/self_orienting_{}.csv'.format(game_type))

In [18]:
for game in ["change_agent_game"]: #"contingency_game", "contingency_game_shuffled_1", 
    get_self_finding_steps(game)

In [5]:
# Read all keep_close control prop data, average it and save it in a csv file
files = glob.glob("../data/change_agent_game_harder/keep_close_control_prop/*.json")

all_data = {i: [] for i in range(0, 54)}
for i, file in enumerate(files):  # Get the data for each participant in that level
    data = json.load(open(file))

    for lvl in data.keys():
        all_data[int(lvl)].append(data[lvl])

# Average the data
avg_data = {lvl: statistics.mean(all_data[lvl]) for lvl in all_data.keys()}

# Save the data in a csv file
df = pd.DataFrame({'level': avg_data.keys(), 'keep_close_control_prop': avg_data.values()})
df.to_csv('../stats/keep_close_control_prop.csv')